In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer
from textblob import Word 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB


In [17]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")

In [60]:
# Create textblob objects of the tweets
sentiment_objects = [TextBlob(tw) for tw in train['text']]
# Create list of polarity valuesx and tweet text
sentiment_values = [[tweet.sentiment.polarity] for tweet in sentiment_objects]

#Values closer to 1 indicate more positivity, while values closer to -1 indicate more negativity.
sentiment_df = pd.DataFrame(sentiment_values, columns=["polarity"])

In [ ]:
def add_polarity(_df):
    _df = pd.concat([_df, sentiment_df], axis=1)
    return _df
train=add_polarity(train)

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

In [ ]:
# Cleaning & Preprocessing tweets Data
stop_words = set(stopwords.words('english'))
tweet_1=[]
for tw in train["text"]:
    word_tokens = word_tokenize(tw) 
    #Delete ponctuation
    word_tokens=[word.lower() for word in word_tokens if word.isalpha()]
    #Delete stop words
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    filtered_sentence = [] 
  
    for w in word_tokens: 
        if w not in stop_words : 
            if  w!='http':
                filtered_sentence.append(w) 

    Stem_words = []
    ps =PorterStemmer()
    for w in filtered_sentence:
        rootWord=ps.stem(w)
        Stem_words.append(rootWord)
    lem=[]
    for w in filtered_sentence:
        word1 = Word(w).lemmatize("n")
        word2 = Word(word1).lemmatize("v")
        word3 = Word(word2).lemmatize("a")
        lem.append(Word(word3).lemmatize())
    tweet_1.append(lem)

In [ ]:
sentiment_objects = [TextBlob(str(t)) for t in tweet_1]
sentiment_values = [[tweet_1.sentiment.polarity, str(tweet_1)] for tweet_1 in sentiment_objects]
sentiment_values[0]
sentiment_df1 = pd.DataFrame(sentiment_values, columns=["polarity_lem", "lems"])

In [18]:
def add_polarity1(_df):
    _df = pd.concat([_df, sentiment_df1["lems"]], axis=1)
    return _df

In [19]:
train=add_polarity1(train)

In [ ]:
train["lems"]= train["lems"].str.replace("[", "") 
train["lems"]= train["lems"].str.replace("]", "") 
train["lems"]= train["lems"].str.replace("\'", "") 
train["lems"]= train["lems"].str.replace(",", " ") 

In [ ]:
#Cleaning & Preprocessing tweets Data
stop_words = set(stopwords.words('english'))
tweet_2=[]
for tw in test["text"]:
    word_tokens = word_tokenize(tw) 
    #Delete ponctuation
    word_tokens=[word.lower() for word in word_tokens if word.isalpha()]
    #Delete stop words
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    filtered_sentence = [] 
  
    for w in word_tokens: 
        if w not in stop_words : 
            if  w!='http':
                filtered_sentence.append(w) 
    
    Stem_words = []
    ps =PorterStemmer()
    for w in filtered_sentence:
        rootWord=ps.stem(w)
        Stem_words.append(rootWord)
    lem=[]
    for w in filtered_sentence:
        word1 = Word(w).lemmatize("n")
        word2 = Word(word1).lemmatize("v")
        word3 = Word(word2).lemmatize("a")
        lem.append(Word(word3).lemmatize())
    tweet_2.append(lem)

In [ ]:
sentiment_objects = [TextBlob(str(t)) for t in tweet_2]
sentiment_values = [[tweet_2.sentiment.polarity, str(tweet_2)] for tweet_2 in sentiment_objects]
sentiment_values[0]
sentiment_df1 = pd.DataFrame(sentiment_values, columns=["polarity_lem", "lems"])

In [ ]:
def add_polarity2(_df):
    _df = pd.concat([_df, sentiment_df1], axis=1)
    return _df
test=add_polarity2(test)

In [ ]:
test["lems"]= test["lems"].str.replace("[", "") 
test["lems"]= test["lems"].str.replace("]", "") 
test["lems"]= test["lems"].str.replace("\'", "") 
test["lems"]= test["lems"].str.replace(",", " ") 

In [ ]:
count_vectorizer = CountVectorizer()
X_train = count_vectorizer.fit_transform(train["lems"])
X_test = count_vectorizer.transform(test["lems"])

In [ ]:
#TfidfVectorizer - Convert text to word frequency vectors.
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(train["lems"])
X_test_tfidf = tfidf_vectorizer.transform(test["lems"])

In [62]:
## Our vectors are really big, so we want to push our model's weights
clf = linear_model.RidgeClassifier()

In [63]:
scores = model_selection.cross_val_score(clf, X_train, train["target"], cv=3, scoring="f1")

In [ ]:
clf.fit(X_train,train["target"])

In [87]:
# Fitting Logistic Regression on Count Vectors
logistic_reg= LogisticRegression(solver='liblinear')
scores = model_selection.cross_val_score(logistic_reg, X_train, train["target"], cv=3, scoring="f1")

In [66]:
logistic_reg.fit(X_train,train["target"])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [86]:
# Fitting Logistic Regression on tfidf
logistic_reg_tfidf = LogisticRegression(solver='liblinear')
scores = model_selection.cross_val_score(logistic_reg_tfidf, X_train_tfidf, train["target"], cv=3, scoring="f1")

logistic_reg_tfidf.fit(X_train_tfidf,train["target"])
y_predicted_logistic_reg_tfidf = logistic_reg_tfidf.predict(X_test_tfidf)

In [69]:
multinomial_naive_bayes = MultinomialNB()
scores = model_selection.cross_val_score(multinomial_naive_bayes, X_train, train["target"], cv=3, scoring="f1")

In [70]:
multinomial_naive_bayes.fit( X_train, train["target"])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [71]:
# Fitting Naive Bayes on tfidf
multinomial_naive_bayes_tfidf = MultinomialNB()
scores = model_selection.cross_val_score(multinomial_naive_bayes_tfidf, X_train_tfidf, train["target"], cv=3, scoring="f1")
multinomial_naive_bayes_tfidf.fit(X_train_tfidf, train["target"])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [84]:
# Fitting SVM on Count Vector
svd = TruncatedSVD(n_components=120)
xtrain_svd = svd.fit_transform(X_train)
xtest_svd= svd.transform(X_test)

# Scale the data obtained from SVD. Renaming variable to reuse without scaling.
scl = StandardScaler()
xtrain_svd_scl = scl.fit_transform(xtrain_svd)
xtest_svd_scl = scl.transform(xtest_svd)
# Fitting a simple SVM
svm = SVC(gamma='auto')
scores = model_selection.cross_val_score(svm , xtrain_svd_scl, train["target"], cv=3, scoring="f1")
svm.fit(xtrain_svd_scl,  train["target"])

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [85]:
# Fitting SVM on tfidf
svd_tfidf = TruncatedSVD(n_components=120)
xtrain_svd_tfidf = svd_tfidf.fit_transform(X_train_tfidf)
xtest_svd_tfidf = svd_tfidf.transform(X_test_tfidf)

# Scale the data obtained from SVD. Renaming variable to reuse without scaling.
scl_tfidf = StandardScaler()
xtrain_svd_scl_tfidf = scl_tfidf.fit_transform(xtrain_svd_tfidf)
xtest_svd_scl_tfidf = scl_tfidf.transform(xtest_svd_tfidf)

# Fitting a simple SVM
svm_tfidf = SVC(gamma='auto')
scores = model_selection.cross_val_score(svm_tfidf , xtrain_svd_scl_tfidf, train["target"], cv=3, scoring="f1")
svm_tfidf.fit(xtrain_svd_scl_tfidf,  train["target"])

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [74]:
# Fitting xgboost on Count Vector
xgb_classifier = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, subsample=0.8, nthread=10, learning_rate=0.1)
scores = model_selection.cross_val_score(xgb_classifier, X_train, train["target"], cv=3, scoring="f1")
xgb_classifier.fit(X_train,  train["target"])

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=200,
       n_jobs=1, nthread=10, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.8)

In [75]:
# Fitting xgboost on tfidf
xgb_classifier_tfidf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, subsample=0.8, nthread=10, learning_rate=0.1)
scores = model_selection.cross_val_score(xgb_classifier_tfidf , X_train_tfidf, train["target"], cv=3, scoring="f1")
xgb_classifier_tfidf.fit(X_train_tfidf,train["target"])

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=200,
       n_jobs=1, nthread=10, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.8)

In [76]:
# Fitting xgboost on Count Vector svd feature
xgb_classifier_svd= xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, subsample=0.8, nthread=10, learning_rate=0.1)
scores = model_selection.cross_val_score(xgb_classifier_svd, xtrain_svd, train["target"], cv=3, scoring="f1")
xgb_classifier_svd.fit(xtrain_svd, train["target"])

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=200,
       n_jobs=1, nthread=10, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.8)

In [77]:
# Fitting a simple xgboost on tfidf svd features
xgb_classifier_svd_tfidf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, subsample=0.8, nthread=10, learning_rate=0.1)
scores = model_selection.cross_val_score(xgb_classifier_svd_tfidf, xtrain_svd_scl_tfidf, train["target"], cv=3, scoring="f1")
xgb_classifier_svd_tfidf.fit(xtrain_svd_scl_tfidf, train["target"])

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=200,
       n_jobs=1, nthread=10, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.8)

In [78]:
sample_submission = pd.read_csv("sample_submission.csv")

In [79]:
sample_submission["target"]= multinomial_naive_bayes.predict(X_test)

In [80]:
sample_submission.to_csv("submission.csv", index=False)

In [81]:
sample_submission_test = pd.read_csv("test.csv")

In [82]:
sample_submission_test["target"]= multinomial_naive_bayes.predict(X_test)

In [83]:
sample_submission_test.to_csv("submission_test.csv", index=False)